In [1]:
import requests
import json
import pandas as pd

fdi_key = {
    # Request headers of fund daily info
    'Ocp-Apim-Subscription-Key': 'afbbb604d2814b59b0e5fa6ff3717e16'
}

ffs_key = {
    # Request headers of fund fact sheet
    'Ocp-Apim-Subscription-Key': '16c86c3eb3cb4fee8f54932b92630aef'
}

In [2]:
list_rmf_path = 'C:/Users/momot/Documents/Mahidol/Senior project/Finance-Made-Easy-Personalized-Planner/airflow/config/rmf.txt'
with open(list_rmf_path, 'r') as file:
    list_rmf = [line.strip() for line in file]

print(list_rmf)

['M0049_2565', 'M0074_2544', 'M0079_2544', 'M0080_2544', 'M0081_2544', 'M0088_2565', 'M0146_2565', 'M0181_2564', 'M0236_2554', 'M0295_2564', 'M0310_2564', 'M0317_2565', 'M0374_2565', 'M0424_2562', 'M0425_2562', 'M0426_2562', 'M0457_2565', 'M0511_2558', 'M0518_2558', 'M0522_2558', 'M0547_2564', 'M0548_2564', 'M0555_2564', 'M0570_2560', 'M0593_2558', 'M0612_2558', 'M0721_2565', 'M0778_2554', 'M0787_2554', 'M0933_2554', 'M0935_2554']


In [1]:
from datetime import date, timedelta
from datetime import datetime

# ไม่ได้ดึงวันนี้ แต่ดึงถึง วันนี้ - 1
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

def convert_string_to_datetime(date_string:str)-> datetime:
    date_object = datetime.strptime(date_string, '%Y-%m-%d')
    return date_object

## (non-dynamic) initial แต่ละ csv ดึง YTD

In [16]:
end_date = datetime.now()

In [22]:
import csv
import time
header = ['nav_date',
'net_asset',
'last_val',
'sell_price',
'buy_price',
'sell_swap_price',
'buy_swap_price']
for rmf_id in list_rmf:
    year =  rmf_id.rsplit("_", 1) #Split _ to dictate years in Thai Calendar, TH year - 543 = A.D., B Year
    start_year = (int)(year[1])-543 #At index 1, in list String year is Year in TH, then we subtract by 543
    start_date = datetime(start_year,1,1)
    filename = f'../config/data/rmf_nav_data/'+rmf_id+'_RMF.csv'
    # writemode = 'w' เขียนทับ
    with open(filename, 'w',newline = '') as f_out:
        f_out = csv.writer(f_out,quoting=csv.QUOTE_ALL)
        f_out.writerow(header)
        for date in daterange(start_date,end_date):
                res_FDI = requests.get(f'https://api.sec.or.th/FundDailyInfo/{rmf_id}/dailynav/{date.strftime("%Y-%m-%d")}', headers = fdi_key)
                time.sleep(0.5)
                print(res_FDI)
                print(date)
                if(res_FDI.status_code == 200):
                    
                    res_json = json.loads(res_FDI.text)
                    print(res_json)
                    # break
                    try:
                        f_out.writerow([res_json['nav_date'],
                                        res_json['net_asset'],
                                        res_json['last_val'],
                                        res_json['amc_info'][0]['sell_price'],
                                        res_json['amc_info'][0]['buy_price'],
                                        res_json['amc_info'][0]['sell_swap_price'],
                                        res_json['amc_info'][0]['buy_swap_price']],
                                        )
                    except Exception as e:
                         pass

<Response [204]>
2022-01-01 00:00:00
<Response [204]>
2022-01-02 00:00:00
<Response [204]>
2022-01-03 00:00:00
<Response [204]>
2022-01-04 00:00:00
<Response [204]>
2022-01-05 00:00:00
<Response [204]>
2022-01-06 00:00:00
<Response [204]>
2022-01-07 00:00:00
<Response [204]>
2022-01-08 00:00:00
<Response [204]>
2022-01-09 00:00:00
<Response [204]>
2022-01-10 00:00:00
<Response [204]>
2022-01-11 00:00:00
<Response [204]>
2022-01-12 00:00:00
<Response [204]>
2022-01-13 00:00:00
<Response [204]>
2022-01-14 00:00:00
<Response [204]>
2022-01-15 00:00:00
<Response [204]>
2022-01-16 00:00:00
<Response [204]>
2022-01-17 00:00:00
<Response [204]>
2022-01-18 00:00:00
<Response [204]>
2022-01-19 00:00:00
<Response [204]>
2022-01-20 00:00:00
<Response [204]>
2022-01-21 00:00:00
<Response [204]>
2022-01-22 00:00:00
<Response [204]>
2022-01-23 00:00:00
<Response [204]>
2022-01-24 00:00:00
<Response [204]>
2022-01-25 00:00:00
<Response [204]>
2022-01-26 00:00:00
<Response [204]>
2022-01-27 00:00:00
<

## (dynamic) ดึงวันที่ของ row สุดท้ายใน csv จนถึงเมื่อวาน

In [2]:
from datetime import date, timedelta
from datetime import datetime

# ไม่ได้ดึงวันนี้ แต่ดึงถึง วันนี้ - 1
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

def convert_string_to_datetime(date_string:str)-> datetime:
    date_object = datetime.strptime(date_string, '%Y-%m-%d')
    return date_object

In [31]:
list_rmf_path = 'C:/Users/momot/Documents/Mahidol/Senior project/Finance-Made-Easy-Personalized-Planner/airflow/config/rmf.txt'
with open(list_rmf_path, 'r') as file:
    list_rmf = [line.strip() for line in file]
print(list_rmf)

['M0074_2544', 'M0079_2544', 'M0080_2544', 'M0081_2544', 'M0088_2565', 'M0146_2565', 'M0181_2564', 'M0295_2564', 'M0310_2564', 'M0317_2565', 'M0424_2562', 'M0425_2562', 'M0426_2562', 'M0511_2558', 'M0518_2558', 'M0522_2558', 'M0547_2564', 'M0548_2564', 'M0555_2564', 'M0570_2560', 'M0593_2558', 'M0778_2554', 'M0787_2554', 'M0933_2554', 'M0935_2554']


In [6]:
end_date = datetime.now()

In [4]:
# Check deprecated file
import pandas as pd
import os

list_rmf_path = 'C:/Users/momot/Documents/Mahidol/Senior project/Finance-Made-Easy-Personalized-Planner/airflow/config/data/rmf_nav_data/'
for filename in os.listdir(list_rmf_path):
    df = pd.read_csv(list_rmf_path+filename)
    # print(df.tail(1))
    date = df.tail(1)['nav_date'].to_string(index=False)
    if date =='Series([], )':
        print(filename)
    # print(date)

In [4]:
import csv
import time
import pandas as pd

header = ['nav_date',
'net_asset',
'last_val',
'sell_price',
'buy_price',
'sell_swap_price',
'buy_swap_price']

for rmf_id in list_rmf:
    year =  rmf_id.rsplit("_", 1) #Split _ to dictate years in Thai Calendar, TH year - 543 = A.D., B Year
    start_year = (int)(year[1])-543 #At index 1, in list String year is Year in TH, then we subtract by 543
    start_date = datetime(start_year,1,1)
    filename = f'../config/data/rmf_nav_data/'+rmf_id+'_RMF.csv'
    # writemode = 'w' เขียนทับ
    with open(filename, 'w',newline = '') as f_out:
        f_out = csv.writer(f_out,quoting=csv.QUOTE_ALL)
        f_out.writerow(header)
        for date in daterange(start_date,end_date):
                res_FDI = requests.get(f'https://api.sec.or.th/FundDailyInfo/{rmf_id}/dailynav/{date.strftime("%Y-%m-%d")}', headers = fdi_key)
                time.sleep(0.2)
                print(res_FDI)
                print(date)
                if(res_FDI.status_code == 200):
                    res_json = json.loads(res_FDI.text)
                    f_out.writerow([res_json['nav_date'],
                                    res_json['net_asset'],
                                    res_json['last_val'],
                                    res_json['amc_info'][0]['sell_price'],
                                    res_json['amc_info'][0]['buy_price'],
                                    res_json['amc_info'][0]['sell_swap_price'],
                                    res_json['amc_info'][0]['buy_swap_price']],
                                    )

NameError: name 'end_date' is not defined

## ดึง URL Factsheet ของทุกกองทุน RMF ภายใต้ SCB เพื่อหาระดับความเสี่ยง

In [32]:
import requests
import json
import pandas as pd
import time
import csv

fdi_key = {
    # Request headers of fund daily info
    'Ocp-Apim-Subscription-Key': 'afbbb604d2814b59b0e5fa6ff3717e16'
}

ffs_key = {
    # Request headers of fund fact sheet
    'Ocp-Apim-Subscription-Key': '16c86c3eb3cb4fee8f54932b92630aef'
}

In [33]:
list_rmf_path = 'C:/Users/momot/Documents/Mahidol/Senior project/Finance-Made-Easy-Personalized-Planner/airflow/config/rmf.txt'
with open(list_rmf_path, 'r') as file:
    list_rmf = [line.strip() for line in file]
print(list_rmf)

['M0074_2544', 'M0079_2544', 'M0080_2544', 'M0081_2544', 'M0088_2565', 'M0146_2565', 'M0181_2564', 'M0295_2564', 'M0310_2564', 'M0317_2565', 'M0424_2562', 'M0425_2562', 'M0426_2562', 'M0511_2558', 'M0518_2558', 'M0522_2558', 'M0547_2564', 'M0548_2564', 'M0555_2564', 'M0570_2560', 'M0593_2558', 'M0778_2554', 'M0787_2554', 'M0933_2554', 'M0935_2554']


In [34]:
df = pd.DataFrame()
for rmf_id in list_rmf:
    # print(filename)
    data_dict = {'rmf_id': ''
                 ,'url_factsheet':''}

    response = requests.get(f'https://api.sec.or.th/FundFactsheet/fund/{rmf_id}/URLs', headers=ffs_key)
    response_json = json.loads(response.text)

    # Create dictionary
    data_dict['rmf_id'] = rmf_id
    data_dict['url_factsheet'] = response_json['url_factsheet']
    # print(type(data_dict))
    api_df = pd.DataFrame(data_dict, index=[0])

    # Append the data to main Dataframe
    df = pd.concat([df, api_df])

    time.sleep(0.2)

df.to_csv('url_factsheet.csv', header=True, index=False)